## Implementing Sentence Embedding RAG with Gemma Chatbot

### Definition
RAG, or Retriever-Augmented Generation, is a model that enhances response generation by first retrieving relevant information from a large dataset and then using that context to generate detailed and accurate answers. This approach allows the model to access additional information from documentation without needing finetuning and reduces the rate of hallucinations.

<p align="center">
  <img src="https://assets-global.website-files.com/63f3993d10c2a062a4c9f13c/64593ba041a4ff8dfef73f30_1*LYApKuxzzmvFECqwYk61wg.png" title="Image source: https://www.ml6.eu/blogpost/leveraging-llms-on-your-domain-specific-knowledge-base">
</p>

### System
**Database**: We construct a database from additional documents about Data Science. Large documents are divided into chunks of a certain size, and an embedding is generated from each chunk. This approach allows us to represent the whole document with a single vector. The vectors are then stored in a vector store; we use FAISS, a library for efficient similarity search that is based on [HNSW](https://towardsdatascience.com/similarity-search-part-4-hierarchical-navigable-small-world-hnsw-2aad4fe87d37).

**Information Search**: Given the user query, we create a sentence embedding from it and compare it with each chunk using cosine similarity. Then, either the top k documents or those that pass a certain similarity threshold are retrieved.

**Reply Generation**: The retrieved documents are then passed to an LLM with a prompt to answer the question with the helpful retrieved information. Because the potential answer is given within the prompt, we don't need a powerful Language Model with a huge number of parameters but rather use the 2B Gemini model.

### Technology
- **Sentence Transformers**: For accessing pretrained models.
- **LangChain**: For constructing the pipeline.
- **FAISS**: For storing the vector database.

### More about RAG
- [RAG with Langchain tutorial](https://towardsdatascience.com/retrieval-augmented-generation-rag-from-theory-to-langchain-implementation-4e9bd5f6a4f2)
- [Stanford lecture](https://www.youtube.com/watch?v=mE7IDf2SmJg)


In [1]:
# install required libraries
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install langchain
!pip install sentence-transformers
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.0.11 requires j

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [3]:
class Assistant:
    """Gemma 2b based assistant that replies given the retrieved documents"""
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/2b-it/2")
        self.Gemma = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/2b-it/2")

    def create_prompt(self, query, retrieved_info):
        # instruction to areply to query given the retrived information
        prompt = f"""You need either to explain the concept or answer the question about Datta Science. 
        Be detailed, use simple words and examples in your explanations. If required, utilize the relevant information.
        Instruction: {query}
        Relevant information: {retrieved_info}
        Output:
        """
        return prompt
    
    def reply(self, query, retrieved_info):
        prompt = self.create_prompt(query, retrieved_info)
        input_ids = self.tokenizer(query, return_tensors="pt").input_ids
        # Generate text with a focus on factual responses
        generated_text = self.Gemma.generate(
            input_ids,
            max_length=500, # let answers be not that long
            temperature=0.7, # Adjust temperature according to the task, for code generation it can be 0.9
        )
        # Decode and return the answer
        answer = self.tokenizer.decode(generated_text[0], skip_special_tokens=True)
        return answer


In [4]:
import os
def get_all_pdfs(directory):
    """Get the list of pdf files in the directory."""
    pdf_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files


class Retriever:
    """Sentence embedding based Retrieval Based Augmented generation.
        Given database of pdf files, retriever finds num_retrieved_docs relevant documents"""
    def __init__(self, num_retrieved_docs=5, pdf_folder_path='/kaggle/input/data-science-cheat-sheets/Data Science'):
        # load documents
        pdf_files = get_all_pdfs(pdf_folder_path)
        print("Documents used", pdf_files)
        loaders = [PyPDFLoader(pdf_file) for pdf_file in pdf_files]
        all_documents = []
        for loader in loaders:
            raw_documents = loader.load()
            # split the documents into smaller chunks
            text_splitter = CharacterTextSplitter( 
                separator="\n\n",
                chunk_size=800,
                chunk_overlap=100,
                length_function=len,
            )
            documents = text_splitter.split_documents(raw_documents)
            all_documents.extend(documents)
        # create a vectorstore database
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") # fast model with competitive perfomance
        self.db = FAISS.from_documents(all_documents, embeddings)
        self.retriever = self.db.as_retriever(search_kwargs={"k": num_retrieved_docs})

    def search(self, query):
        # retrieve top k similar documents to query
        docs = self.retriever.get_relevant_documents(query)
        return docs

In [5]:
chatbot = Assistant()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
retriever = Retriever()

Documents used ['/kaggle/input/data-science-cheat-sheets/Data Science/Going Pro in Data Science .pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/Data Driven Creating a Data Culture .pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/Handbook_Pt2.pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/The-Ultimate-Guide-to-Effective-Data-Collection.pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/Ultimate Guide to Data Cleaning.pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/Handbook_Pt1.pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/Feature Engineering.pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/Cheat Sheets for AI, Neural Networks, Machine Learning, Deep Learning, Big Data.pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/Data Science Cheat Sheet(Python_R).pdf', '/kaggle/input/data-science-cheat-sheets/Data Science/The 5 Feature Selection Algorithms every Data Scientist should know.pdf', '/kaggle/in

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
def generate_reply(query):
    related_docs = retriever.search(query)
    #print('related docs', related_docs)
    reply = chatbot.reply(query, related_docs)
    return reply

In [8]:
# example
reply = generate_reply("Explain Data Cleaning")
for s in reply.split('\n'):
    print(s)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Explain Data Cleaning and Data Transformation

**Data Cleaning**

* **Identifying and correcting errors:** This involves identifying and correcting mistakes, inconsistencies, and missing values in the data.
* **Handling outliers:** Outliers are data points that are significantly different from the rest of the data. Outliers can be handled by removing them, transforming them, or using them for analysis.
* **Normalizing data:** Data can be normalized to ensure that it is on a consistent scale. Normalization can help to improve the performance of machine learning algorithms.

**Data Transformation**

* **Data aggregation:** Data aggregation is the process of combining data points into a single record. For example, data can be aggregated by calculating the mean, median, or standard deviation of a set of values.
* **Data discretization:** Data discretization is the process of dividing data into discrete categories. For example, data can be discretized by dividing it into categories based on